# Transactions Database
Explore the Transactions Database using Python + SQL in the SCx Package

In [1]:
pip install scx>=1.2.1 #Note: This may throw an Error in Google Colab since Colab uses an outdated duckdb package. This file should still work in Colab without issues.

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import the Database class
from scx.database import Database
# Import the pretty print module
from pprint import pprint

In [3]:
# Specify the S3 path to the data
data_folder = 's3://scx-dev/databases/supermarket/'
# Create the database
db = Database(f"""
    CREATE TABLE Customers AS SELECT * FROM read_parquet('{data_folder}customers.parquet');
    CREATE TABLE CustomerSurvey AS SELECT * FROM read_parquet('{data_folder}customer_survey.parquet');
    CREATE TABLE Items AS SELECT * FROM read_parquet('{data_folder}items.parquet');
    CREATE TABLE Sales AS SELECT * FROM read_parquet('{data_folder}sales.parquet');
    CREATE TABLE Stores AS SELECT * FROM read_parquet('{data_folder}stores.parquet');
    CREATE TABLE Transactions AS SELECT * FROM read_parquet('{data_folder}transactions.parquet');
    CREATE TABLE ZipData AS SELECT * FROM read_parquet('{data_folder}zipdata.parquet');
""")

In [4]:
# Show the Schema
db.show_info()

Table: CustomerSurvey
  Customer_ID: NUMBER
  Cust_Sex: STRING
  Cust_Income: NUMBER
  Cust_Race: STRING
  Cust_Age: STRING
  Cust_Children: STRING
  Cust_Rel_Status: STRING

Table: Customers
  Customer_ID: NUMBER
  First_Name: STRING
  Last_Name: STRING
  Customer_Address: STRING
  Customer_City: STRING
  Customer_State: STRING
  Customer_Zip: NUMBER
  Customer_Phone_Number: STRING

Table: Items
  Item_ID: NUMBER
  Item_Name: STRING
  Price_Per_Item: NUMBER

Table: Sales
  Sale_ID: NUMBER
  Customer_ID: NUMBER
  Store_ID: NUMBER
  Sale_Week: NUMBER

Table: Stores
  Store_ID: NUMBER
  Store_Name: STRING
  Store_Size: STRING
  Store_Address: STRING
  Store_City: STRING
  Store_State: STRING
  Store_Zip: NUMBER
  Store_Phone_Number: STRING

Table: Transactions
  Sale_ID: NUMBER
  Item_ID: NUMBER
  Amount_Purchased: NUMBER
  Item_Discount: NUMBER

Table: ZipData
  zip_code: NUMBER
  City: STRING
  State: STRING
  median_income: STRING
  N_Households: NUMBER
  Pct_White: NUMBER
  Pct_Black

In [5]:
# Return the most expensive item in the Items table
pprint(db.query("""
    SELECT * FROM Items
    WHERE Price_Per_Item = (SELECT MAX(Price_Per_Item) FROM Items)
    LIMIT 1;
"""))

[{'Item_ID': 799673, 'Item_Name': 'Cake', 'Price_Per_Item': 9.14}]


In [6]:
# Select the first 5 rows of the Transactions table and join it 
# with the Items table to get the:
# - Sale_ID
# - Cost (Amount_Purchased * Price_Per_Item * (1-Item_Discount))
#   - Round the Cost to 2 decimal places
# - Item_Name
# - Item_Quantity
pprint(db.query("""
    SELECT 
        Transactions.Sale_ID,
        Round(
            Transactions.Amount_Purchased * Items.Price_Per_Item * (1-Transactions.Item_Discount)
            ,2
        ) AS Cost,
        Items.Item_Name,
        Transactions.Amount_Purchased AS Item_Quantity
    FROM 
        Transactions, Items
    WHERE
        Transactions.Item_ID = Items.Item_ID
    LIMIT 5;   
"""))

[{'Cost': 23.4, 'Item_Name': 'Watermelons', 'Item_Quantity': 4, 'Sale_ID': 1},
 {'Cost': 7.49, 'Item_Name': 'Blackberries', 'Item_Quantity': 2, 'Sale_ID': 1},
 {'Cost': 3.52, 'Item_Name': 'Beets', 'Item_Quantity': 1, 'Sale_ID': 1},
 {'Cost': 7.88, 'Item_Name': 'Bell Pepper', 'Item_Quantity': 2, 'Sale_ID': 1},
 {'Cost': 3.74, 'Item_Name': 'Broccoli', 'Item_Quantity': 1, 'Sale_ID': 1}]


In [7]:
# Select the total cost of each sale in the Transactions table
# Round the total cost to 2 decimal places
# Show only the first 5 rows

pprint(db.query("""
    SELECT 
        Transactions.Sale_ID,
        ROUND(
            SUM(
                Transactions.Amount_Purchased * 
                Items.Price_Per_Item * 
                (1-Transactions.Item_Discount)
            )
            ,2
        ) AS Total_Cost
    FROM
        Transactions, Items
    WHERE
        Transactions.Item_ID = Items.Item_ID
    GROUP BY
        Transactions.Sale_ID
    LIMIT 5;    
"""))

[{'Sale_ID': 28818, 'Total_Cost': 205.18},
 {'Sale_ID': 28837, 'Total_Cost': 190.11},
 {'Sale_ID': 28840, 'Total_Cost': 187.67},
 {'Sale_ID': 28851, 'Total_Cost': 180.58},
 {'Sale_ID': 28855, 'Total_Cost': 268.32}]


In [8]:
# Show the total sales for each store in the database
# You should return the Store_Name and the Total_Sales
# Round the total sales to 2 decimal places

pprint(db.query("""
    SELECT
        Stores.Store_Name,
        ROUND(
            SUM(
                Transactions.Amount_Purchased * 
                Items.Price_Per_Item * 
                (1-Transactions.Item_Discount)
            )
            ,2
        ) AS Total_Sales
    FROM
        Sales, Transactions, Items, Stores
    WHERE
        Sales.Sale_ID = Transactions.Sale_ID
        AND Transactions.Item_ID = Items.Item_ID
        AND Sales.Store_ID = Stores.Store_ID
    GROUP BY
        Sales.Store_ID, Stores.Store_Name
    ORDER BY
        Total_Sales DESC;
"""))

[{'Store_Name': "Brian's Bazaar", 'Total_Sales': 849936.02},
 {'Store_Name': "Sally's Shop", 'Total_Sales': 824521.79},
 {'Store_Name': "Eddy's Exchange", 'Total_Sales': 720931.45},
 {'Store_Name': "Bill's Barter", 'Total_Sales': 638185.34},
 {'Store_Name': "Chris's Corner", 'Total_Sales': 431861.6},
 {'Store_Name': "Marty's Mart", 'Total_Sales': 372514.57},
 {'Store_Name': "Mary's Market", 'Total_Sales': 362563.64},
 {'Store_Name': "Delio's Deli", 'Total_Sales': 340409.11},
 {'Store_Name': "Ahmad's Alley", 'Total_Sales': 339852.21},
 {'Store_Name': "Eva's Extravaganza", 'Total_Sales': 296243.6},
 {'Store_Name': "Emma's Emporium", 'Total_Sales': 195622.17},
 {'Store_Name': "Sheffi's Store", 'Total_Sales': 102756.9}]


In [9]:
# Show top 5 customers by total spend
# You should return the Customer_Name, and Total_Spend
# Use a WITH statement to create a temporary table to calculate the cost of each transaction

pprint(db.query("""
    WITH TransactionCost AS (
        SELECT
            Transactions.Sale_ID,
            SUM(
                Transactions.Amount_Purchased * 
                Items.Price_Per_Item * 
                (1-Transactions.Item_Discount)
            ) AS Cost
        FROM
            Transactions, Items
        WHERE
            Transactions.Item_ID = Items.Item_ID
        GROUP BY
            Transactions.Sale_ID
    )      
    SELECT
        Customers.First_Name,
        Customers.Last_Name,
        ROUND(SUM(TransactionCost.Cost),2) AS Total_Spend
    FROM
        TransactionCost, Sales, Customers
    WHERE
        TransactionCost.Sale_ID = Sales.Sale_ID
    AND
        Sales.Customer_ID = Customers.Customer_ID
    GROUP BY
        Customers.Customer_ID, Customers.First_Name, Customers.Last_Name
    ORDER BY
        Total_Spend DESC
    LIMIT 5;
"""))

[{'First_Name': 'Melody', 'Last_Name': 'Bunch', 'Total_Spend': 5085.33},
 {'First_Name': 'Delores', 'Last_Name': 'Mahoney', 'Total_Spend': 5056.23},
 {'First_Name': 'Daniel', 'Last_Name': 'Hawkins', 'Total_Spend': 4957.21},
 {'First_Name': 'Dong', 'Last_Name': 'Rasmussen', 'Total_Spend': 4890.16},
 {'First_Name': 'Rogelio', 'Last_Name': 'Schmidt', 'Total_Spend': 4599.15}]
